# RNA-seq Allelic Imbalance Analysis with WASP2

This tutorial demonstrates a complete workflow for detecting allele-specific expression (ASE) in bulk RNA-seq data using WASP2.

**Estimated time:** ~30 minutes

## Overview

We will cover:
1. **Data Loading** - BAM, VCF, and gene annotations
2. **Allele Counting** - Count reads at heterozygous SNPs within genes
3. **Statistical Testing** - Beta-binomial model for allelic imbalance
4. **ASE Visualization** - Visualize results
5. **Imprinting Detection** - Identify monoallelic expression patterns
6. **eQTL Integration** - Connect to regulatory variant databases

## Prerequisites

**Software:**
- WASP2 (`pip install wasp2`)
- Python packages: pandas, numpy, matplotlib, seaborn

**Data:**
- Aligned BAM file (coordinate-sorted, indexed)
- Phased VCF file with heterozygous variants
- Gene annotation file (GTF format)

## Setup and Imports

In [ ]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Configure plotting style
sns.set_style("whitegrid")
sns.set_palette("colorblind")
warnings.filterwarnings("ignore")

# Display settings
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 200)

## Section 1: Data Loading

### Input Files

For this tutorial, you'll need:

| File | Description | Example |
|------|-------------|--------|
| BAM | Aligned RNA-seq reads | `sample.bam` |
| VCF | Phased variant calls | `variants.vcf.gz` |
| GTF | Gene annotations | `genes.gtf` |

### Define Input Paths

In [ ]:
# Configure your input files here
# Replace these paths with your actual data

BAM_FILE = "sample.bam"  # Your aligned BAM file
VCF_FILE = "variants.vcf.gz"  # Your phased VCF file
GTF_FILE = "genes.gtf"  # Gene annotation (e.g., GENCODE)
SAMPLE_ID = "SAMPLE1"  # Sample name in VCF

# Output directory
OUTPUT_DIR = Path("rnaseq_results")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"BAM file: {BAM_FILE}")
print(f"VCF file: {VCF_FILE}")
print(f"GTF file: {GTF_FILE}")
print(f"Sample ID: {SAMPLE_ID}")
print(f"Output directory: {OUTPUT_DIR}")

### Verify Input Files

Check that all required files exist and have the correct format.

In [ ]:
def verify_inputs(bam: str, vcf: str, gtf: str) -> bool:
    """Verify input files exist and check for BAM index.

    Note: This function only checks file existence, not format validation.
    """
    all_ok = True

    # Check BAM file and index
    if not Path(bam).exists():
        print(f"ERROR: BAM file not found: {bam}")
        all_ok = False
    elif not Path(f"{bam}.bai").exists() and not Path(bam.replace(".bam", ".bai")).exists():
        print(f"WARNING: BAM index not found. Run: samtools index {bam}")
    else:
        print(f"OK: BAM file found: {bam}")

    # Check VCF file
    if not Path(vcf).exists():
        print(f"ERROR: VCF file not found: {vcf}")
        all_ok = False
    else:
        print(f"OK: VCF file found: {vcf}")

    # Check GTF file
    if not Path(gtf).exists():
        print(f"ERROR: GTF file not found: {gtf}")
        all_ok = False
    else:
        print(f"OK: GTF file found: {gtf}")

    return all_ok


# Uncomment to verify your files:
# verify_inputs(BAM_FILE, VCF_FILE, GTF_FILE)

### Preview VCF Data

Check the VCF format and verify phasing information is present.

In [ ]:
def preview_vcf(vcf_file: str, n_lines: int = 5) -> None:
    """Preview VCF header and first few variants."""
    import gzip

    opener = gzip.open if vcf_file.endswith(".gz") else open

    with opener(vcf_file, "rt") as f:
        # Show last few header lines
        header_lines = []
        for line in f:
            if line.startswith("#"):
                header_lines.append(line.strip())
            else:
                break

        print("VCF Header (last 3 lines):")
        for line in header_lines[-3:]:
            print(f"  {line[:100]}..." if len(line) > 100 else f"  {line}")

        # Show first few variants
        print(f"\nFirst {n_lines} variants:")
        f.seek(0)
        count = 0
        for line in f:
            if not line.startswith("#"):
                fields = line.strip().split("\t")
                chrom, pos, _, ref, alt = fields[:5]
                gt = fields[9].split(":")[0] if len(fields) > 9 else "N/A"
                print(f"  {chrom}:{pos} {ref}>{alt} GT={gt}")
                count += 1
                if count >= n_lines:
                    break

        # Check phasing
        is_phased = "|" in gt
        print(f"\nPhasing detected: {'YES' if is_phased else 'NO (unphased)'}")


# Uncomment to preview your VCF:
# preview_vcf(VCF_FILE)

## Section 2: Allele Counting at Genes

WASP2 counts reads supporting each allele at heterozygous SNP positions, annotated with overlapping genes.

### Run Allele Counting

The `wasp2-count count-variants` command:
1. Identifies heterozygous SNPs from the VCF
2. Counts reads supporting reference vs alternate alleles
3. Annotates each SNP with the overlapping gene from the GTF

In [ ]:
# Output file for allele counts
COUNTS_FILE = OUTPUT_DIR / "allele_counts.tsv"

# Build the wasp2-count command
count_cmd = f"""
wasp2-count count-variants \
    {BAM_FILE} \
    {VCF_FILE} \
    --samples {SAMPLE_ID} \
    --region {GTF_FILE} \
    --out_file {COUNTS_FILE}
"""

print("Allele counting command:")
print(count_cmd)

In [ ]:
# Execute the counting command
# Uncomment to run:
# result = subprocess.run(count_cmd.strip(), shell=True, capture_output=True, text=True)
# if result.returncode == 0:
#     print("Allele counting completed successfully!")
# else:
#     print(f"Error: {result.stderr}")

### Examine Count Data

The output TSV contains allele counts per SNP, annotated with gene information.

In [ ]:
# Load and examine count data
# For demonstration, we'll create example data
# Replace with: counts_df = pd.read_csv(COUNTS_FILE, sep='\t')

# Example count data structure
example_counts = pd.DataFrame(
    {
        "chr": ["chr1", "chr1", "chr1", "chr2", "chr2"],
        "pos": [100000, 150000, 200000, 50000, 75000],
        "ref": ["A", "G", "C", "T", "A"],
        "alt": ["G", "A", "T", "C", "G"],
        "ref_count": [45, 23, 156, 89, 34],
        "alt_count": [52, 78, 42, 91, 67],
        "other_count": [1, 0, 2, 0, 1],
        "gene_id": ["ENSG00000001", "ENSG00000001", "ENSG00000002", "ENSG00000003", "ENSG00000003"],
        "gene_name": ["GENE1", "GENE1", "GENE2", "GENE3", "GENE3"],
        f"{SAMPLE_ID}": ["0|1", "0|1", "1|0", "0|1", "0|1"],
    }
)

print("Count data structure:")
print(example_counts.head())
print(f"\nShape: {example_counts.shape}")
print(f"Unique genes: {example_counts['gene_id'].nunique()}")

### Gene-Level Summary

Aggregate SNP-level counts to gene-level for analysis.

In [ ]:
def summarize_by_gene(counts_df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate allele counts per gene."""
    gene_summary = (
        counts_df.groupby(["gene_id", "gene_name"])
        .agg(
            {
                "ref_count": "sum",
                "alt_count": "sum",
                "pos": "count",  # Number of SNPs per gene
            }
        )
        .rename(columns={"pos": "n_snps"})
        .reset_index()
    )

    # Calculate total counts and allele ratio
    gene_summary["total_count"] = gene_summary["ref_count"] + gene_summary["alt_count"]
    gene_summary["ref_ratio"] = gene_summary["ref_count"] / gene_summary["total_count"]

    return gene_summary


gene_summary = summarize_by_gene(example_counts)
print("Gene-level summary:")
print(gene_summary)

## Section 3: Statistical Testing

WASP2 uses a beta-binomial model to test for significant deviation from 50:50 allele ratios.

### Why Beta-Binomial?

The beta-binomial model accounts for:
- **Overdispersion**: Biological variation beyond binomial sampling
- **Technical noise**: PCR amplification, sequencing errors
- **Multiple SNPs**: Aggregating information across SNPs in a gene

### Run Allelic Imbalance Analysis

In [ ]:
# Output file for imbalance results
AI_RESULTS_FILE = OUTPUT_DIR / "ai_results.tsv"

# Build the wasp2-analyze command
analyze_cmd = f"""
wasp2-analyze find-imbalance \
    {COUNTS_FILE} \
    --min 10 \
    --pseudocount 1 \
    --phased \
    --out_file {AI_RESULTS_FILE}
"""

print("Analysis command:")
print(analyze_cmd)

In [ ]:
# Execute the analysis command
# Uncomment to run:
# result = subprocess.run(analyze_cmd.strip(), shell=True, capture_output=True, text=True)
# if result.returncode == 0:
#     print("Analysis completed successfully!")
# else:
#     print(f"Error: {result.stderr}")

### Understanding the Output

The output includes:

| Column | Description |
|--------|-------------|
| `region` | Gene or region identifier |
| `ref_count` | Total reference allele counts |
| `alt_count` | Total alternate allele counts |
| `p_value` | Likelihood ratio test p-value |
| `fdr_pval` | FDR-corrected p-value |
| `effect_size` | Log2 fold change (ref/alt) |

In [ ]:
# Example analysis results
# Replace with: results_df = pd.read_csv(AI_RESULTS_FILE, sep='\t')

example_results = pd.DataFrame(
    {
        "region": ["ENSG00000001", "ENSG00000002", "ENSG00000003", "ENSG00000004", "ENSG00000005"],
        "gene_name": ["SNRPN", "H19", "KCNQ1OT1", "ACTB", "GAPDH"],
        "ref_count": [245, 15, 8, 156, 189],
        "alt_count": [12, 234, 287, 148, 195],
        "p_value": [1.2e-45, 3.4e-38, 5.6e-52, 0.65, 0.82],
        "fdr_pval": [6.0e-44, 8.5e-37, 5.6e-50, 0.75, 0.85],
        "effect_size": [4.35, -3.96, -5.16, 0.08, -0.05],
        "dispersion": [0.02, 0.03, 0.01, 0.05, 0.04],
    }
)

print("Allelic imbalance results:")
print(example_results)

### Filter Significant Results

In [ ]:
# Filter by significance threshold
FDR_THRESHOLD = 0.05
EFFECT_SIZE_THRESHOLD = 1.0  # log2 fold change

significant = example_results[
    (example_results["fdr_pval"] < FDR_THRESHOLD)
    & (abs(example_results["effect_size"]) > EFFECT_SIZE_THRESHOLD)
]

print(f"Significant ASE genes (FDR < {FDR_THRESHOLD}, |log2FC| > {EFFECT_SIZE_THRESHOLD}):")
print(f"Found {len(significant)} genes with significant allelic imbalance\n")
print(significant[["gene_name", "ref_count", "alt_count", "effect_size", "fdr_pval"]])

## Section 4: ASE Visualization

### Volcano Plot

Visualize effect size vs significance to identify strong ASE signals.

In [ ]:
def plot_volcano(results: pd.DataFrame, fdr_thresh: float = 0.05) -> plt.Figure:
    """Create a volcano plot for ASE results."""
    fig, ax = plt.subplots(figsize=(10, 8))

    # Calculate -log10 p-value
    results = results.copy()
    results["-log10_pval"] = -np.log10(results["p_value"] + 1e-300)

    # Classify points
    is_sig = results["fdr_pval"] < fdr_thresh

    # Plot non-significant
    ax.scatter(
        results.loc[~is_sig, "effect_size"],
        results.loc[~is_sig, "-log10_pval"],
        c="gray",
        alpha=0.5,
        s=20,
        label="Not significant",
    )

    # Plot significant
    ax.scatter(
        results.loc[is_sig, "effect_size"],
        results.loc[is_sig, "-log10_pval"],
        c="red",
        alpha=0.7,
        s=40,
        label=f"FDR < {fdr_thresh}",
    )

    # Add gene labels for significant hits
    for _, row in results[is_sig].iterrows():
        ax.annotate(
            row["gene_name"],
            (row["effect_size"], row["-log10_pval"]),
            fontsize=8,
            ha="left",
            va="bottom",
        )

    # Add reference lines
    ax.axhline(-np.log10(0.05), color="black", linestyle="--", alpha=0.3)
    ax.axvline(0, color="black", linestyle="-", alpha=0.3)
    ax.axvline(1, color="blue", linestyle=":", alpha=0.3)
    ax.axvline(-1, color="blue", linestyle=":", alpha=0.3)

    ax.set_xlabel("Effect Size (log2 Ref/Alt)", fontsize=12)
    ax.set_ylabel("-log10(p-value)", fontsize=12)
    ax.set_title("Allele-Specific Expression Volcano Plot", fontsize=14)
    ax.legend()

    plt.tight_layout()
    return fig


fig = plot_volcano(example_results)
# fig.savefig(OUTPUT_DIR / 'volcano_plot.png', dpi=150)
plt.show()

### Allele Ratio Distribution

In [ ]:
def plot_allele_distribution(results: pd.DataFrame) -> plt.Figure:
    """Plot distribution of reference allele ratios."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Calculate ref ratio
    results = results.copy()
    results["ref_ratio"] = results["ref_count"] / (results["ref_count"] + results["alt_count"])

    # Histogram
    axes[0].hist(results["ref_ratio"], bins=30, edgecolor="white", alpha=0.7)
    axes[0].axvline(0.5, color="red", linestyle="--", label="Expected (0.5)")
    axes[0].set_xlabel("Reference Allele Ratio")
    axes[0].set_ylabel("Number of Genes")
    axes[0].set_title("Distribution of Allele Ratios")
    axes[0].legend()

    # Box plot by significance
    results["significant"] = results["fdr_pval"] < 0.05
    results["status"] = results["significant"].map({True: "Significant", False: "Not Significant"})

    colors = {"Significant": "coral", "Not Significant": "lightblue"}
    for status, color in colors.items():
        data = results[results["status"] == status]["ref_ratio"]
        bp = axes[1].boxplot(
            [data], positions=[list(colors.keys()).index(status)], patch_artist=True, widths=0.6
        )
        bp["boxes"][0].set_facecolor(color)

    axes[1].axhline(0.5, color="red", linestyle="--", alpha=0.5)
    axes[1].set_xticks([0, 1])
    axes[1].set_xticklabels(["Significant", "Not Significant"])
    axes[1].set_ylabel("Reference Allele Ratio")
    axes[1].set_title("Allele Ratio by Significance")

    plt.tight_layout()
    return fig


fig = plot_allele_distribution(example_results)
plt.show()

## Section 5: Imprinting and Monoallelic Expression Detection

Genomic imprinting results in parent-of-origin-specific gene expression. Imprinted genes show extreme allelic imbalance (>90:10 or <10:90).

### Identify Candidate Imprinted Genes

In [ ]:
def detect_monoallelic(results: pd.DataFrame, ratio_threshold: float = 0.9) -> pd.DataFrame:
    """
    Identify genes with monoallelic expression patterns.

    Parameters
    ----------
    results : pd.DataFrame
        ASE results with ref_count and alt_count columns
    ratio_threshold : float
        Threshold for monoallelic classification (default: 0.9 means >90:10)

    Returns
    -------
    pd.DataFrame
        Filtered results with monoallelic genes and classification
    """
    results = results.copy()

    # Calculate allele ratios
    total = results["ref_count"] + results["alt_count"]
    results["ref_ratio"] = results["ref_count"] / total
    results["alt_ratio"] = results["alt_count"] / total

    # Classify expression pattern
    conditions = [
        results["ref_ratio"] >= ratio_threshold,
        results["alt_ratio"] >= ratio_threshold,
    ]
    choices = ["Ref-monoallelic", "Alt-monoallelic"]
    results["pattern"] = np.select(conditions, choices, default="Biallelic")

    # Filter to monoallelic genes with significant p-values
    monoallelic = results[(results["pattern"] != "Biallelic") & (results["fdr_pval"] < 0.05)].copy()

    return monoallelic


# Detect monoallelic expression (using 0.9 threshold for >90:10 ratio)
monoallelic_genes = detect_monoallelic(example_results, ratio_threshold=0.9)

print(f"Monoallelic genes detected: {len(monoallelic_genes)}")
if len(monoallelic_genes) > 0:
    print(monoallelic_genes[["gene_name", "ref_ratio", "pattern", "fdr_pval"]])

### Compare to Known Imprinted Genes

Cross-reference detected monoallelic genes with known imprinted gene databases.

In [ ]:
# Known imprinted genes in humans (partial list from Geneimprint database)
KNOWN_IMPRINTED_GENES = {
    # Maternally expressed
    "H19",
    "MEG3",
    "MEG8",
    "CDKN1C",
    "PHLDA2",
    "SLC22A18",
    # Paternally expressed
    "IGF2",
    "SNRPN",
    "SNURF",
    "NDN",
    "MAGEL2",
    "MKRN3",
    "KCNQ1OT1",
    "PEG3",
    "PEG10",
    "MEST",
    "PLAGL1",
    "DLK1",
    "RTL1",
}


def annotate_imprinting(results: pd.DataFrame, known_genes: set) -> pd.DataFrame:
    """Annotate genes with known imprinting status."""
    results = results.copy()
    results["known_imprinted"] = results["gene_name"].isin(known_genes)
    return results


# Annotate results
annotated = annotate_imprinting(example_results, KNOWN_IMPRINTED_GENES)

# Check known imprinted genes in our data
known_in_data = annotated[annotated["known_imprinted"]]
print(f"Known imprinted genes in dataset: {len(known_in_data)}")
if len(known_in_data) > 0:
    print(known_in_data[["gene_name", "effect_size", "fdr_pval", "known_imprinted"]])

### Visualize Imprinting Patterns

In [ ]:
def plot_imprinting_heatmap(results: pd.DataFrame, top_n: int = 20) -> plt.Figure:
    """Plot heatmap of allele ratios for top ASE genes."""
    # Sort by absolute effect size and take top genes
    results = results.copy()
    results["abs_effect"] = abs(results["effect_size"])
    top_genes = results.nlargest(top_n, "abs_effect")

    # Calculate allele ratios
    total = top_genes["ref_count"] + top_genes["alt_count"]
    top_genes["ref_ratio"] = top_genes["ref_count"] / total

    # Create figure
    fig, ax = plt.subplots(figsize=(8, max(6, len(top_genes) * 0.4)))

    # Plot horizontal bar chart
    y_pos = np.arange(len(top_genes))
    colors = ["coral" if r > 0.5 else "steelblue" for r in top_genes["ref_ratio"]]

    bars = ax.barh(y_pos, top_genes["ref_ratio"] - 0.5, left=0.5, color=colors, alpha=0.7)

    # Add reference line
    ax.axvline(0.5, color="black", linestyle="-", linewidth=2)

    # Customize
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_genes["gene_name"])
    ax.set_xlabel("Reference Allele Ratio")
    ax.set_xlim(0, 1)
    ax.set_title("Top ASE Genes by Effect Size")

    # Mark known imprinted genes
    for i, (_, row) in enumerate(top_genes.iterrows()):
        if row["gene_name"] in KNOWN_IMPRINTED_GENES:
            ax.annotate("*", (0.02, i), fontsize=14, color="gold", fontweight="bold")

    plt.tight_layout()
    return fig


fig = plot_imprinting_heatmap(example_results)
plt.show()

## Section 6: eQTL Integration

Expression quantitative trait loci (eQTLs) are genetic variants that influence gene expression. Integrating ASE data with eQTL databases helps identify causal regulatory variants.

### Load eQTL Data

We'll demonstrate integration with GTEx eQTL data. You can download tissue-specific eQTL results from the [GTEx Portal](https://gtexportal.org/).

In [ ]:
# Example eQTL data structure (GTEx format)
# Replace with your actual eQTL data
example_eqtl = pd.DataFrame(
    {
        "gene_id": ["ENSG00000001", "ENSG00000002", "ENSG00000003", "ENSG00000006"],
        "gene_name": ["GENE1", "GENE2", "GENE3", "GENE6"],
        "variant_id": ["rs12345", "rs23456", "rs34567", "rs67890"],
        "tss_distance": [1000, -5000, 25000, 100],
        "pval_nominal": [1e-8, 1e-6, 1e-10, 1e-4],
        "slope": [0.45, -0.32, 0.58, 0.12],
        "tissue": ["Whole_Blood", "Whole_Blood", "Whole_Blood", "Whole_Blood"],
    }
)

print("Example eQTL data:")
print(example_eqtl)

### Integrate ASE with eQTL

In [ ]:
def integrate_eqtl(ase_results: pd.DataFrame, eqtl_data: pd.DataFrame) -> pd.DataFrame:
    """
    Integrate ASE results with eQTL data.

    Parameters
    ----------
    ase_results : pd.DataFrame
        ASE analysis results
    eqtl_data : pd.DataFrame
        eQTL associations (e.g., from GTEx)

    Returns
    -------
    pd.DataFrame
        Merged data with ASE and eQTL information
    """
    # Merge on gene identifier
    merged = ase_results.merge(
        eqtl_data[["gene_id", "variant_id", "pval_nominal", "slope", "tss_distance"]],
        left_on="region",
        right_on="gene_id",
        how="left",
    )

    # Flag genes with eQTL support
    merged["has_eqtl"] = ~merged["variant_id"].isna()

    return merged


# Integrate data
integrated = integrate_eqtl(example_results, example_eqtl)

print("Integrated ASE + eQTL data:")
print(integrated[["gene_name", "effect_size", "fdr_pval", "has_eqtl", "variant_id"]].head())

### Concordance Analysis

Check if ASE direction agrees with eQTL effect direction.

**Understanding Direction Concordance:**
- **ASE effect_size > 0**: Reference allele is MORE expressed
- **eQTL slope > 0**: Alternate allele INCREASES expression (standard eQTL convention)
- **Concordance**: These directions should be OPPOSITE (ref high in ASE = alt low in eQTL)

This counterintuitive relationship occurs because ASE measures relative allele expression while eQTL slopes measure how the alternate allele affects total expression.

In [ ]:
def check_concordance(integrated_data: pd.DataFrame) -> pd.DataFrame:
    """
    Check concordance between ASE and eQTL effect directions.

    Concordance indicates the ASE signal may be driven by the eQTL variant.

    Note on direction interpretation:
    - ASE effect_size > 0 means REFERENCE allele is more expressed
    - eQTL slope > 0 means ALTERNATE allele increases expression
    - Therefore, concordance = OPPOSITE signs (ref high in ASE = alt low in eQTL)
    """
    data = integrated_data.copy()

    # Only analyze genes with both ASE and eQTL data
    has_both = data["has_eqtl"] & (data["fdr_pval"] < 0.05)
    concordance_data = data[has_both].copy()

    if len(concordance_data) == 0:
        print("No genes with both significant ASE and eQTL data.")
        return data

    # Determine effect directions
    concordance_data["ase_direction"] = np.sign(concordance_data["effect_size"])
    concordance_data["eqtl_direction"] = np.sign(concordance_data["slope"])

    # Concordant if directions are OPPOSITE:
    # - ASE effect_size > 0 (ref allele higher) should pair with
    # - eQTL slope < 0 (alt allele decreases expression, i.e., ref higher)
    concordance_data["concordant"] = (
        concordance_data["ase_direction"] != concordance_data["eqtl_direction"]
    )

    n_concordant = concordance_data["concordant"].sum()
    n_total = len(concordance_data)

    print("Concordance analysis:")
    print(f"  Genes with ASE + eQTL: {n_total}")
    print(f"  Concordant direction: {n_concordant} ({n_concordant / n_total * 100:.1f}%)")

    return concordance_data


concordance = check_concordance(integrated)
if len(concordance) > 0:
    print(concordance[["gene_name", "effect_size", "slope", "concordant"]])

### Visualization: ASE vs eQTL Effect

In [ ]:
def plot_ase_eqtl_correlation(integrated_data: pd.DataFrame) -> plt.Figure:
    """Plot correlation between ASE effect size and eQTL slope."""
    fig, ax = plt.subplots(figsize=(8, 8))

    # Filter to genes with both measurements
    data = integrated_data[integrated_data["has_eqtl"]].copy()

    if len(data) == 0:
        ax.text(0.5, 0.5, "No overlapping genes", ha="center", va="center")
        return fig

    # Color by ASE significance
    colors = ["red" if p < 0.05 else "gray" for p in data["fdr_pval"]]

    ax.scatter(data["slope"], data["effect_size"], c=colors, alpha=0.6, s=50)

    # Add labels for significant genes
    for _, row in data[data["fdr_pval"] < 0.05].iterrows():
        ax.annotate(row["gene_name"], (row["slope"], row["effect_size"]), fontsize=8, ha="left")

    # Reference lines
    ax.axhline(0, color="black", linestyle="-", alpha=0.3)
    ax.axvline(0, color="black", linestyle="-", alpha=0.3)

    ax.set_xlabel("eQTL Effect (slope)", fontsize=12)
    ax.set_ylabel("ASE Effect (log2 Ref/Alt)", fontsize=12)
    ax.set_title("ASE vs eQTL Effect Direction", fontsize=14)

    plt.tight_layout()
    return fig


fig = plot_ase_eqtl_correlation(integrated)
plt.show()

## Summary

In this tutorial, we covered:

1. **Data Loading**: Preparing BAM, VCF, and GTF files for analysis
2. **Allele Counting**: Using `wasp2-count count-variants` to count allele-specific reads
3. **Statistical Testing**: Beta-binomial model for detecting significant allelic imbalance
4. **ASE Visualization**: Volcano plots and allele ratio distributions
5. **Imprinting Detection**: Identifying monoallelic expression patterns
6. **eQTL Integration**: Connecting ASE signals to regulatory variants

### Key Takeaways

- **FDR < 0.05** is a common significance threshold for ASE
- **|log2FC| > 1** indicates strong allelic imbalance (2-fold difference)
- **Monoallelic expression** (>90:10 ratio) may indicate imprinting
- **eQTL concordance** helps identify causal regulatory variants

### Next Steps

- Validate top ASE genes with allele-specific qPCR
- Integrate with chromatin accessibility data (ATAC-seq)
- Perform tissue-specific comparisons
- Investigate disease-associated variants in ASE genes

## References

1. **WASP2**: Allele-specific analysis toolkit
2. **GTEx Consortium** (2020). The GTEx Consortium atlas of genetic regulatory effects across human tissues. *Science*
3. **Geneimprint Database**: https://www.geneimprint.com/
4. **Beta-binomial model**: Skelly et al. (2011). A powerful and flexible statistical framework for testing hypotheses of allele-specific gene expression from RNA-seq data. *Genome Research*